In [1]:

import pandas as pd
from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import describe
from thefuzz import fuzz


from knowledge_propagation.modules.evaluators import (
    ExactMatchEvaluator,
    RougeEvaluator,
    OpenAIEvaluator,
)
llm_evaluator = OpenAIEvaluator()

os.getcwd()

'/datastor1/zliu/mend/notebooks'

In [7]:
fpath = "/u/zliu/datastor1/mend/drop_exp_output/llama3.2-1B-eos-sft/drop/base_n=100_prompt=no_w-gen_wo-icl.xlsx"
do_gen = "w-gen" in fpath
df = pd.read_excel(fpath)
metrics = ["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"]
if do_gen:
    metrics += ["rouge1", "llm_accuracy"]
# macro_averaging(df, multi_levelz_averaging=["stage", "question"], metrics=['[Q][A] acc', "rouge1", "llm_accuracy"])
len(df)


1753

In [4]:
save_dir = "/u/zliu/datastor1/mend/drop_exp_output/Llama-3.2-1B-eos-sft_clm-baseline_lr=1e-05_epoch=4.0/individual_results"
all_df = pd.concat([pd.read_excel(f) for f in glob(f"{save_dir}/*.xlsx", recursive=True)])
all_df.to_excel(f"{save_dir}/../all_results_e.xlsx")
print(len(all_df))
cpt_ids = all_df["id"].unique()

1753


In [8]:
(macro_averaging(df[df["id"].isin(cpt_ids)], multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["rouge1", "llm_accuracy"]) * 100).round(1)

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,rouge1,llm_accuracy
0,efficacyefficacyefficacyefficacyefficacyeffica...,8.3,59.4,0.0,27.7,12.8,24.0


In [9]:
(macro_averaging(all_df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["rouge1", "llm_accuracy"]) * 100).round(1)

KeyError: "[('llm_accuracy', 'mean')] not in index"

In [49]:
import math
one_K_all_df = pd.read_excel(f"{save_dir}/../all_table.xlsx")

describe([
    len(vars.GPT_4_TOKENIZER(ans))
    for ans in one_K_all_df["predicted_answer"].to_list() # [one_K_all_df["question_type"] == "single_hop_efficacy"]
    if isinstance(ans, str)
])

DescribeResult(nobs=2920, minmax=(np.int64(1), np.int64(22)), mean=np.float64(16.71027397260274), variance=np.float64(33.4864308944234), skewness=np.float64(-1.423181597382786), kurtosis=np.float64(0.37896054426006387))

NameError: name 'q_df' is not defined

In [24]:
len(vars.GPT_4_TOKENIZER(one_K_all_df["predicted_answer"][0]))

2

In [38]:
math.isnan(one_K_all_df["predicted_answer"].to_list()[7])

True

# Split single-hop questions

In [18]:
id2single_hops = {x['id']: x["single_hop_efficacy"] for x in io.load_jsonlines(f"{vars.DATA_DIR}/musique_mend_converted/2hop_musique_ans_v1.0_dev.jsonl")[:1000]}

In [19]:
id2single_hops["2hop__108336_772156"]

[{'id': 108336,
  'question': "Which was the creator of allegory of isabella d'este's coronation?",
  'answer': 'Lorenzo Costa',
  'supporting_text_id': 0},
 {'id': 772156,
  'question': 'Where did lorenzo costa pass away according to his place of death?',
  'answer': 'Mantua',
  'supporting_text_id': 1}]

In [4]:
# {x['id']: x["multi_hop_efficacy"] for x in io.load_jsonlines(f"{vars.DATA_DIR}/musique_mend_converted/2hop_musique_ans_v1.0_dev.jsonl")[:1000]}

In [20]:
df_path = "/u/zliu/datastor1/mend/exp_output/Llama-3.2-1B-eos-sft_clm-baseline_input=seen-hop_lr=1e-05_epoch=4.0/all_table_e+s.xlsx"

df = pd.read_excel(df_path)

df_multiplier = 2 if "pre-edit" in df["stage"].tolist() and "post-edit" in df["stage"].tolist() else 1

In [24]:
instance_df

,id,question_type,clm_input,stage,question,answer,predicted_answer_idx,predicted_answer,exact_match,rouge1,rouge2,rougeL,rougeLsum,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,llm_accuracy
2792,2hop__10017_18974,single_hop_specificity,[[In 677 (during the reign of Emperor Gaozong)...,post-edit,Which ocean lies to the east of Australia?,Pacific Ocean,0,Indian Ocean,0,0.500000,0.0,0.500000,0.500000,0,0.666667,0,0.416667,0.1
2793,2hop__10017_18974,single_hop_specificity,[[In 677 (during the reign of Emperor Gaozong)...,post-edit,Who was the first person to land on the moon?,Neil Armstrong,0,Neil Armstrong,1,1.000000,1.0,1.000000,1.000000,1,1.000000,0,0.714286,1.0
2794,2hop__10017_18974,single_hop_specificity,[[In 677 (during the reign of Emperor Gaozong)...,post-edit,Which country is known as the Land of the Risi...,Japan,0,Japan,1,1.000000,0.0,1.000000,1.000000,1,1.000000,0,0.428571,1.0
2795,2hop__10017_18974,single_hop_specificity,[[In 677 (during the reign of Emperor Gaozong)...,post-edit,Who developed the theory of relativity?,Albert Einstein,0,Albert Einstein,1,1.000000,1.0,1.000000,1.000000,1,1.000000,0,0.636364,1.0
2796,2hop__10017_18974,multi_hop_specificity,[[In 677 (during the reign of Emperor Gaozong)...,post-edit,Which ocean lies to the east of the continent ...,Atlantic Ocean,0,Pacific Ocean,0,0.500000,0.0,0.500000,0.500000,0,0.666667,0,0.388889,0.1
2797,2hop__10017_18974,multi_hop_specificity,[[In 677 (during the reign of Emperor Gaozong)...,post-edit,"Which country, known as the Land of the Rising...",Japan,0,Japan,1,1.000000,0.0,1.000000,1.000000,1,1.000000,0,0.392857,1.0
2798,2hop__10017_18974,single_hop_efficacy,[[In 677 (during the reign of Emperor Gaozong)...,post-edit,Who led the military expedition in Fujian ?,Chen Zheng,0,"Zhang Zheng (陳政), together with his son Zhang ...",0,0.181818,0.0,0.181818,0.181818,0,0.333333,0,0.333333,0.3
2799,2hop__10017_18974,multi_hop_efficacy,[[In 677 (during the reign of Emperor Gaozong)...,post-edit,Who led the military expedition in the provinc...,Chen Zheng,0,"Zhang Zheng (陳政), together with his son Zhang ...",0,0.181818,0.0,0.181818,0.181818,0,0.333333,0,0.200000,0.3


In [22]:
new_df = []
for ins_id, instance_df in df.groupby("id"):
    # assert len(instance_df) == 3 * df_multiplier, ins_id
    single_hop_df = instance_df[instance_df["question_type"] == "single_hop_efficacy"]
    assert len(single_hop_df) == 2 * df_multiplier
    
    for r_i, row in single_hop_df.iterrows():
        
        row_dict = row.to_dict()
        try:
            question_lst = [q["question"] for q in id2single_hops[ins_id]]
            q_index: int = int(np.argmax([fuzz.ratio(q, row_dict["question"]) for q in question_lst]))
        except:
            print(ins_id)
            assert False
        row_dict["question_tag"] = f"single_hop_efficacy_q{q_index}"
        new_df.append(row_dict)

    multi_hop_df = instance_df[instance_df["question_type"] == "multi_hop_efficacy"]
    assert len(multi_hop_df) == 1 * df_multiplier
    
    for r_i, row in multi_hop_df.iterrows():
        
        row_dict = row.to_dict()
        row_dict["question_tag"] = f"multi_hop_efficacy_q0"
        new_df.append(row_dict)


AssertionError: 

In [11]:
pd.DataFrame(new_df).to_excel(io.remove_last_extension(df_path) + "_split.xlsx", index=False)

ModuleNotFoundError: No module named 'thefuzz'

In [108]:
for r_i, row in single_hop_df.iterrows():
    q_index: int = [q["question"] for q in id2single_hops[ins_id]].index(row["question"])
    row["question_tag"] = f"single_hop_efficacy_q{q_index}"
    pass

In [109]:
row

Unnamed: 0                                                              1
id                                                        2hop__998_25855
question_type                                         single_hop_efficacy
clm_input               [[The third generation began including a 30-pi...
stage                                                           post-edit
question                What does dell call the feature that lets USB ...
answer                                                         PowerShare
predicted_answer_idx                                                    0
predicted_answer                                                    Sleep
exact_match                                                             0
rouge1                                                                0.0
rouge2                                                                0.0
rougeL                                                                0.0
rougeLsum                             

1

'What does dell call the feature that lets USB drives to remain powered when the computer is off?'

In [22]:
df1 = pd.read_excel("/u/zliu/datastor1/mend/exp_output/Llama-3.2-1B-eos-sft_clm-baseline_input=first-1hop_lr=1e-05_epoch=4.0/all_table.xlsx")
df2 = pd.read_excel("/u/zliu/datastor1/mend/exp_output/Llama-3.2-1B-eos-sft_clm-baseline_input=second-1hop_lr=1e-05_epoch=4.0/all_table.xlsx")
df = pd.concat([df1, df2])

In [23]:
df[df["stage"] == "post-edit"].head()

,Unnamed: 0,id,question_type,clm_input,stage,question,answer,predicted_answer_idx,predicted_answer,exact_match,rouge1,rouge2,rougeL,rougeLsum,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,llm_accuracy
0,0,2hop__159903_154896,single_hop_efficacy,[[He was elected to the NLF executive in the f...,post-edit,What part of the country is it about?,South Yemen,0,Northern Ireland,0,0.0,0.0,0.0,0.0,0,0.666667,0,0.416667,NaN
1,1,2hop__159903_154896,single_hop_efficacy,[[He was elected to the NLF executive in the f...,post-edit,What year did South Yemen insurgency start?,27 April 2009,0,1965,0,0.0,0.0,0.0,0.0,0,0.571429,0,0.333333,NaN
2,2,2hop__159903_154896,multi_hop_efficacy,[[He was elected to the NLF executive in the f...,post-edit,In what year did the insurgency start in the p...,27 April 2009,0,1965,0,0.0,0.0,0.0,0.0,0,0.428571,0,0.375000,NaN
3,0,2hop__341305_332933,single_hop_efficacy,[[Pine Springs is an unincorporated community ...,post-edit,In which administrative territorial entity is ...,Culberson County,0,"The state of Texas, United States. Pine Spring...",0,0.0,0.0,0.0,0.0,0,0.600000,0,0.200000,0.7
4,1,2hop__341305_332933,single_hop_efficacy,[[Pine Springs is an unincorporated community ...,post-edit,What is the capital of culberson county?,Van Horn,0,"Fort Worth, Texas, United States. It is the cl...",0,0.0,0.0,0.0,0.0,0,0.333333,0,0.538462,0.1


In [24]:
len(df[df["stage"] == "post-edit"])

6000

In [25]:
macro_averaging(df[df["stage"] == "post-edit"], multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["rouge1", "llm_accuracy"]) * 100

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,rouge1,llm_accuracy
0,multi_hop_efficacymulti_hop_efficacymulti_hop_...,3.550,52.908916,0.0,37.829996,15.716392,35.1750
1,single_hop_efficacysingle_hop_efficacysingle_h...,8.425,59.864995,0.0,41.268511,23.778972,38.8625


In [27]:
raw_questions = []
for icl_question in df["question"]:
    question_ = icl_question.split("\n")[-2].strip()
    assert "Q: " in question_
    raw_questions.append(question_[3:])
    

6000

In [28]:
raw_questions[:3]

['Who is the performer associated with green?',
 'Who is the spouse of steve hillage?',
 'Who is the spouse of the Green performer?']

In [29]:
scored_df = llm_acc_per_example = llm_evaluator.compute_metric(
    questions=raw_questions,
    predictions=df["predicted_answer"],
    references=df["answer"],
    use_aggregator=False,
    rescale_to_one=True,
)

2025-02-26 21:51:25.107 | INFO     | knowledge_propagation.modules.evaluators:compute_metric:104 - Evaluating with LLM evaluator (gpt-4o-mini)


Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix
Failing heuristic fix


In [37]:
df["llm_accuracy"] = scored_df["llm_accuracy"]

In [41]:
macro_averaging(df[df["stage"] == "pre-edit"], multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["rouge1", "llm_accuracy"]) * 100

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,rouge1,llm_accuracy
0,multi_hop_efficacymulti_hop_efficacymulti_hop_...,1.5,48.632919,0.0,37.569110,5.807457,16.170
1,single_hop_efficacysingle_hop_efficacysingle_h...,6.2,54.202771,0.0,38.640035,12.080476,23.095


In [40]:
macro_averaging(df[df["stage"] == "post-edit"], multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["rouge1", "llm_accuracy"]) * 100

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,rouge1,llm_accuracy
0,multi_hop_efficacymulti_hop_efficacymulti_hop_...,7.85,58.005100,0.0,40.070285,15.668783,24.8950
1,single_hop_efficacysingle_hop_efficacysingle_h...,19.40,67.272676,0.0,41.425836,28.166019,35.8125


In [49]:
for x in io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/musique_mend_converted/2hop_musique_ans_v1.0_train.jsonl"):
    assert len(x["multi_hop_efficacy"]) == 1, x
    assert len(x["single_hop_efficacy"]) == 2, x

In [3]:
malmen_combiner_text_df = pd.read_excel("/u/zliu/datastor1/malmen/exp_output/musique_combiner_text_w-atomq_m1/malmen_eval_musique_combiner_text_n=1000.xlsx")
malmen_combiner_text_df = malmen_combiner_text_df[(malmen_combiner_text_df["stage"] == "post-edit") & (malmen_combiner_text_df["question_type"] == "multi_hop_efficacy")]

clm_combiner_text_df = pd.read_excel("/u/zliu/datastor1/mend/exp_output/Llama-3.2-1B-eos-sft_clm-baseline_input=two-1hop_lr=1e-05_epoch=4.0/all_table_split.xlsx")
clm_combiner_text_df = clm_combiner_text_df[(clm_combiner_text_df["stage"] == "post-edit") & (clm_combiner_text_df["question_type"] == "multi_hop_efficacy")]

In [4]:
assert len(malmen_combiner_text_df) == len(clm_combiner_text_df)

In [5]:
c = 0
clm_better_rs = []
for r_i, clm_r in clm_combiner_text_df.iterrows():
    malmen_q_r = malmen_combiner_text_df[malmen_combiner_text_df["question"] == clm_r["question"]]
    if len(malmen_q_r) != 1:
        continue
    malmen_r = malmen_q_r.iloc[0]
    if clm_r["llm_accuracy"] > malmen_r["llm_accuracy"]:
        clm_better_rs.append((clm_r, malmen_r))
    

In [47]:
len(clm_better_rs)

287

In [31]:
with open("/u/zliu/datastor1/mend/spotcheck/clm_compq_better.txt", "w") as f:

    for clm_r, malmen_r in clm_better_rs[:]:
        f.write("=" * 60 + "\n")
        f.write("Example id: " + clm_r["id"] + "\n")
        f.write("Text:\n")
        f.write("-" * 40 + "\n")
        f.write(clm_r["clm_input"] + "\n")
        f.write("-" * 40 + "\n")
        f.write("CompQ: " + clm_r["question"] + "\n")
        f.write("Answer: " + clm_r["answer"] + "\n")
        f.write(f"CLM Prediction [llm_acc={clm_r['llm_accuracy']}]: " + clm_r["predicted_answer"] + "\n")
        f.write(f"MALMEN Prediction [llm_acc={malmen_r['llm_accuracy']}]: " + malmen_r["predicted_answer"] + "\n")
        f.write("=" * 60 + "\n\n")


In [ ]:

baseline = "/u/zliu/datastor1/mend/exp_output/Llama-3.2-1B-eos-sft_sh+mh/all_table_split.xlsx"

1.0

In [35]:
baseline = pd.read_excel("/u/zliu/datastor1/mend/exp_output/Llama-3.2-1B-eos-sft_sh+mh/all_table_split.xlsx")

In [45]:
with open("/u/zliu/datastor1/mend/spotcheck/eos-sft_EMvsRouge.txt", "w") as f:
    f.write("baseline_r[[A]|[Q] Acc EM] == 0 and baseline_r[rouge1]\n")
    interesting_rows = []
    for r_i, baseline_r in baseline.iterrows():
        if baseline_r["[A]|[Q] Acc EM"] == 0 and baseline_r["rouge1"] > 0:
            interesting_rows.append(baseline_r.to_dict())
            f.write("Answer: " + baseline_r["answer"] + "\n")
            f.write("Predicted Answer: " + baseline_r["predicted_answer"] + "\n")
            f.write("\n")
    # if 
    # malmen_q_r = malmen_combiner_text_df[malmen_combiner_text_df["question"] == clm_r["question"]]

{'Unnamed: 0': 2,
 'id': '2hop__998_25855',
 'question_type': 'multi_hop_efficacy',
 'question': 'What does dell call the feature letting the interface replacing FireWire in later iterations of the iPod drives to remain powered when the computer is off?',
 'stage': 'pre-edit',
 'answer': 'PowerShare',
 'predicted_answer_idx': 0,
 'predicted_answer': 'Sleep',
 'exact_match': 0,
 'rouge1': 0.0,
 'rouge2': 0.0,
 'rougeL': 0.0,
 'rougeLsum': 0.0,
 '[A]|[Q] Acc EM': 0,
 '[A]|[Q] Acc PM': 0.3333333432674408,
 '[Q][A] Acc EM': 0,
 '[Q][A] Acc PM': 0.161290317773819,
 'llm_accuracy': 0.1,
 'question_tag': 'multi_hop_efficacy_q0'}

In [2]:
ripple_effect_recent = io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/ripple_edits/benchmark/recent.json")

In [12]:
ripple_effect_recent[0][0].keys()

dict_keys(['example_type', 'edit', 'Logical_Generalization', 'Compositionality_I', 'Compositionality_II', 'Subject_Aliasing', 'Relation_Specificity', 'Forgetfulness'])

In [ ]:
ripple_effect_recent[0][1]["edit"]

{'prompt': 'The place of death of Leo Arons is Berlin.',
 'subject_id': 'Q88641',
 'relation': 'PLACE_OF_DEATH',
 'target_id': 'Q64'}

In [24]:
ripple_effect_recent[0][1]["Subject_Aliasing"]

[{'test_queries': [{'prompt': 'The place of death of Martin Leo Arons is',
    'answers': [{'value': 'Berlin',
      'aliases': ['Berlin, Germany', 'Berlin (Germany)', 'DE-BE']}],
    'query_type': 'regular',
    'subject_id': 'Q88641',
    'relation': 'PLACE_OF_DEATH',
    'target_ids': ['Q64'],
    'phrase': 'The place of death of Martin Leo Arons is'}],
  'test_condition': 'OR',
  'condition_queries': [{'prompt': 'The place of death of Martin Leo Arons is',
    'answers': [{'value': 'Berlin',
      'aliases': ['Berlin, Germany', 'Berlin (Germany)', 'DE-BE']}],
    'query_type': 'regular',
    'subject_id': 'Q88641',
    'relation': 'PLACE_OF_DEATH',
    'target_ids': ['Q64'],
    'phrase': 'The place of death of Martin Leo Arons is'}]}]